In [44]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np
import math

In [45]:
# Расчёт effect_size для пропорций
def calc_proportion_es(prob1: float, prob2: float):
    return abs(proportion_effectsize(prob1, prob2))


# Расчёт effect_size для непрерывных метрик
def calc_continuous_es(mean_control: Union[float, int],
                                std_control: Union[float, int],
                                mean_test: Union[float, int],
                                std_test: Union[float, int]):
    return abs(effectsize_smd(mean_control,
                             std_control,
                             1e4,
                             mean_test,
                             std_test,
                             1e4)[0])

In [46]:
# Расчёт минимально необходимой выборки * 2(для теста и контроля) для пропорций
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)
# Расчёт минимально необходимой выборки * 2(для теста и контроля) для непрерывной метрики
def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

# Задание 1
Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео.

Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156.

Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%.

Средний трафик 20000 человек в день.

Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% .

In [47]:
# вводим данные
tss_control, tss_test = 25, 25 * 1.1 # тестовое значение больше контрольного на 10%
std_control, std_test = 156, 156 # т.к. мы не знаем дисперсию тестовой группы, берем ее равной контрольной
traffic = 20_000

In [48]:
# рассчитываем effect size
es_cont = calc_continuous_es(tss_control, std_control, tss_test, std_test)
es_cont

0.016025039996486808

In [49]:
# параметры alpha и beta при расчете количества испытаний в нашем случае можно не указывать, т.к. они стоят по умолчанию

print(f"При рассчитанном effect size={es_cont}, принимая во внимание значение alpha = 5% и beta = 20%, получаем "
      f"необходимое количество наблюдений равное {calc_sample_size_continuous(es_cont)}."
      f" Учитывая, что средний трафик составляет {traffic} человек в день и необходимое количество наблюдений, получаем,"
      f" что эксперимен должен длиться {math.ceil(calc_sample_size_continuous(es_cont) / traffic)} дней.")

При рассчитанном effect size=0.016025039996486808, принимая во внимание значение alpha = 5% и beta = 20%, получаем необходимое количество наблюдений равное 122257. Учитывая, что средний трафик составляет 20000 человек в день и необходимое количество наблюдений, получаем, что эксперимен должен длиться 7 дней.


# Задание 2
Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта.

Одна из основных метрик bounce rate в GA = 40%.

Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.

Средний трафик 4000 человек в день.

Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%

Исходя из условия, предположу, что bounce rate должна снизиться минимум на 20%

In [50]:
# считаем effectsize
br_control, br_test = 0.4, 0.4 - 0.4 * 0.2
traffic = 4_000

In [51]:
# считаем effectsize
es_prop = calc_proportion_es(br_control, br_test)
es_prop

0.16690997264630925

In [52]:
# параметры alpha и beta при расчете количества испытаний в нашем случае можно не указывать, т.к. они стоят по умолчанию

print(f"При рассчитанном effect size={es_prop}, принимая во внимание значение alpha = 5% и beta = 20%, получаем "
      f"необходимое количество наблюдений равное {calc_sample_size_proportion(es_prop)}."
      f" Учитывая, что средний трафик составляет {traffic} человек в день и необходимое количество наблюдений, получаем,"
      f" что эксперимен должен длиться {math.ceil(calc_sample_size_proportion(es_prop) / traffic)} дней.")

При рассчитанном effect size=0.16690997264630925, принимая во внимание значение alpha = 5% и beta = 20%, получаем необходимое количество наблюдений равное 1126. Учитывая, что средний трафик составляет 4000 человек в день и необходимое количество наблюдений, получаем, что эксперимен должен длиться 1 дней.


Но если допустить, что bounce rate увеличится минимум на 20%, то результат будет примерно такой же:

In [53]:
# считаем effectsize
br_control, br_test = 0.4, 0.4 + 0.4 * 0.2
traffic = 4_000

In [54]:
# считаем effectsize
es_prop = calc_proportion_es(br_control, br_test)
es_prop

0.1613472464363419

In [55]:
# параметры alpha и beta при расчете количества испытаний в нашем случае можно не указывать, т.к. они стоят по умолчанию

print(f"При рассчитанном effect size={es_prop}, принимая во внимание значение alpha = 5% и beta = 20%, получаем "
      f"необходимое количество наблюдений равное {calc_sample_size_proportion(es_prop)}."
      f" Учитывая, что средний трафик составляет {traffic} человек в день и необходимое количество наблюдений, получаем,"
      f" что эксперимен должен длиться {math.ceil(calc_sample_size_proportion(es_prop) / traffic)} дней.")

При рассчитанном effect size=0.1613472464363419, принимая во внимание значение alpha = 5% и beta = 20%, получаем необходимое количество наблюдений равное 1205. Учитывая, что средний трафик составляет 4000 человек в день и необходимое количество наблюдений, получаем, что эксперимен должен длиться 1 дней.


Увеличение или снижение bounce rate минимум на 20% при прочих равных условиях не влияют на продолжительность теста.